# Eigen Value Serial and Parallel code comparison

There are two ways to execute the code.

1.   Paste cuda code directly in colab to run
2.   Execute .cu file via commands

If your running on a hosted python notebook upload the files in the repository to the runtime.

## 1. Paste cuda code directly in colab to run

```
%%cu
<Paste .cu code here>
```

In [1]:
!nvcc --version
!nvidia-smi -L
!lscpu | grep "Model name"

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
GPU 0: Tesla T4 (UUID: GPU-0c701f3a-1967-fe98-fdc3-f366f036c046)
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz


In [2]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-_4c_ak0z
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-_4c_ak0z
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=97ae2fccfb7133824bf33bfd1fca2410ed0a6536d0665e921359130a058bbadf
  Stored in directory: /tmp/pip-ephem-wheel-cache-hnoezbzm/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [4]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [6]:
#!unzip Jacobi_Eigen.zip

Archive:  Jacobi_Eigen.zip
  inflating: data/eigenValues.dat    
  inflating: data/eigenVectors.dat   
  inflating: data/matrix.dat         
  inflating: Jacobi_Eigen_Parallel   
  inflating: Jacobi_Eigen_Parallel.cu  
  inflating: Jacobi_serial_run.sh    
  inflating: JacobiEigen_Serial/Jacobi_Eigen_Serial  
  inflating: JacobiEigen_Serial/Jacobi_Eigen_Serial.cpp  
  inflating: JacobiEigen_Serial/testMatrices.h  
  inflating: JacobiEigen_Serial/utils.h  


In [7]:
%%cu
#include <fstream>
#include <stdio.h> 
#include <iostream>

const int num_mat = 1; // total number of matrices = total number of threads const 
const int N = 20; // square symmetric matrix dimension
const int nTPB = 256; // threads per block

// test symmetric matrices 

double a1[N*N];

/* ---------------------------------------------------------------*/
//****************************************************************************80
//(n, a, d)
__host__ __device__
void r8mat_diag_get_vector(int n, double a[], double v[])
//****************************************************************************80
{
  int i;
  for (i = 0; i < n; i++) {
    v[i] = a[i + i * n];
  }

  return;
}
__host__ __device__
void r8mat_identity(int n, double a[]) {
  int i;
  int j;
  int k;
  k = 0;
  for (j = 0; j < n; j++) {
    for (i = 0; i < n; i++) {
      if (i == j) {
        a[k] = 1.0;
      } else {
        a[k] = 0.0;
      }
      k = k + 1;
    }
  }
  return;
}
//****************************************************************************80

//(n, a + (idx * n * n), it_max, v + (idx * n * n), d + (idx * n), it_num, rot_num)
__host__ __device__
void jacobi_eigenvalue(int n, double a[], int it_max, double v[], double d[], int &it_num, int &rot_num) {
  double * bw;
  double c;
  double g;
  double gapq;
  double h;
  int i;
  int j;
  int k;
  int l;
  int m;
  int p;
  int q;
  double s;
  double t;
  double tau;
  double term;
  double termp;
  double termq;
  double theta;
  double thresh;
  double w;
  double * zw;
  r8mat_identity(n, v);
  r8mat_diag_get_vector(n, a, d);
  bw = new double[n];
  zw = new double[n];
  for (i = 0; i < n; i++) {

    bw[i] = d[i];
    zw[i] = 0.0;
  }
  it_num = 0;
  rot_num = 0;
  while (it_num < it_max) {
    it_num = it_num + 1;
    //
    // The convergence threshold is based on the size of the elements in
    // the strict upper triangle of the matrix.
    //
    thresh = 0.0;
    for (j = 0; j < n; j++) {
      for (i = 0; i < j; i++) {
        thresh = thresh + a[i + j * n] * a[i + j * n];
      }
    }
    thresh = sqrt(thresh) / (double)(4 * n);
    if (thresh == 0.0) {
      break;
    }
    for (p = 0; p < n; p++) {
      for (q = p + 1; q < n; q++) {
        gapq = 10.0 * fabs(a[p + q * n]);
        termp = gapq + fabs(d[p]);
        termq = gapq + fabs(d[q]);
        //
        // Annihilate tiny offdiagonal elements.
        //
        if (4 < it_num &&
          termp == fabs(d[p]) && termq == fabs(d[q])) {
          a[p + q * n] = 0.0;

        }
        //
        // Otherwise, apply a rotation.
        //
        else if (thresh <= fabs(a[p + q * n])) {
          h = d[q] - d[p];
          term = fabs(h) + gapq;
          if (term == fabs(h)) {
            t = a[p + q * n] / h;
          } else {
            theta = 0.5 * h / a[p + q * n];
            t = 1.0 / (fabs(theta) + sqrt(1.0 + theta * theta));
            if (theta < 0.0) {
              t = -t;
            }
          }
          c = 1.0 / sqrt(1.0 + t * t);
          s = t * c;
          tau = s / (1.0 + c);
          h = t * a[p + q * n];
          //
          // Accumulate corrections to diagonal elements.
          //
          zw[p] = zw[p] - h;
          zw[q] = zw[q] + h;
          d[p] = d[p] - h;
          d[q] = d[q] + h;
          a[p + q * n] = 0.0;
          //
          // Rotate, using information from the upper triangle of A only.
          //
          for (j = 0; j < p; j++) {
            g = a[j + p * n];
            h = a[j + q * n];
            a[j + p * n] = g - s * (h + g * tau);
            a[j + q * n] = h + s * (g - h * tau);
          }

          for (j = p + 1; j < q; j++) {
            g = a[p + j * n];
            h = a[j + q * n];
            a[p + j * n] = g - s * (h + g * tau);
            a[j + q * n] = h + s * (g - h * tau);
          }
          for (j = q + 1; j < n; j++) {
            g = a[p + j * n];
            h = a[q + j * n];
            a[p + j * n] = g - s * (h + g * tau);
            a[q + j * n] = h + s * (g - h * tau);
          }
          //
          // Accumulate information in the eigenvector matrix.
          //
          for (j = 0; j < n; j++) {
            g = v[j + p * n];
            h = v[j + q * n];
            v[j + p * n] = g - s * (h + g * tau);
            v[j + q * n] = h + s * (g - h * tau);
          }
          rot_num = rot_num + 1;
        }
      }
    }
    for (i = 0; i < n; i++) {
      bw[i] = bw[i] + zw[i];
      d[i] = bw[i];
      zw[i] = 0.0;
    }
  }
  //
  // Restore upper triangle of input matrix.
  //
  for (j = 0; j < n; j++) {
    for (i = 0; i < j; i++) {
      a[i + j * n] = a[j + i * n];
    }
  }

  //
  // Ascending sort the eigenvalues and eigenvectors.
  //
  for (k = 0; k < n - 1; k++) {
    m = k;
    for (l = k + 1; l < n; l++) {
      if (d[l] < d[m]) {
        m = l;
      }
    }
    if (m != k) {
      t = d[m];
      d[m] = d[k];
      d[k] = t;
      for (i = 0; i < n; i++) {
        w = v[i + m * n];
        v[i + m * n] = v[i + k * n];
        v[i + k * n] = w;
      }
    }
  }
  delete[] bw;
  delete[] zw;
  return;
}

// end of FSU code
/* ---------------------------------------------------------------- */

//(num_mat, N,d_a, max_iter, d_v, d_d)
__global__ void je(int num_matr, int n, double *a, int it_max, double *v, double *d) {
  int idx = threadIdx.x + blockDim.x * blockIdx.x;
  int it_num;
  int rot_num;
  if (idx < num_matr) {
    jacobi_eigenvalue(n, a + (idx * n * n), it_max, v + (idx * n * n), d + (idx * n), it_num, rot_num);
  }
}

//(0, N, a1, h_a)
void initialize_matrix(int mat_id, int n, double * mat, double * v) {
  for (int i = 0; i < n * n; i++) * (v + (mat_id * n * n) + i) = mat[i];
}
void print_vec(int vec_id, int n, double * d) {
  std::cout << "matrix " << vec_id << " Diagonal Values: " << std::endl;
  std::cout.precision(12);
  for (int i = 0; i < n; i++) std::cout << i << ": " << * (d + (n * vec_id) + i) << std::endl;
  std::cout << std::endl;
}
int main() {
  // make sure device heap has enough space for in-kernel new allocations 
  const int heapsize = num_mat * N * sizeof(double) * 2;
  const int chunks = heapsize / (8192 * 1024) + 1;
  cudaError_t cudaStatus = cudaDeviceSetLimit(cudaLimitMallocHeapSize, (8192 * 1024) * chunks);
  if (cudaStatus != cudaSuccess) {
    fprintf(stderr, "set device heap limit failed!");
  }


  //reading matrix from file matrix.dat
  std::ifstream in;
  in.open("data/matrix.dat");
	if (!in)
	{
		std::cerr << "Unable to open file containing matrix. It should have the name matrix.dat. It should be present inside the data folder" << std::endl;
		exit(1);
	}
	else
	{
		for (int i = 0; i < N*N; i++)
		{
			in >> a1[i];
		}
 	  in.close();
  }

  const int max_iter = 1000;
  double *h_a, *d_a, *h_v, *d_v, *h_d, *d_d;

  //initialize_matrix
  h_a = (double * ) malloc(num_mat * N * N * sizeof(double));
  h_v = (double *) malloc(num_mat * N * N * sizeof(double));
  h_d = (double * ) malloc(num_mat * N * sizeof(double));

  //Allocating mem in GPU
  cudaMalloc( & d_a, num_mat * N * N * sizeof(double));
  cudaMalloc( & d_v, num_mat * N * N * sizeof(double));
  cudaMalloc( & d_d, num_mat * N * sizeof(double));

  //sets all the bytes to a specific value (0)
  memset(h_a, 0, num_mat * N * N * sizeof(double));
  memset(h_v, 0, num_mat * N * N * sizeof(double));
  memset(h_d, 0, num_mat * N * sizeof(double));

  for(int i = 0; i< num_mat ; i++)
  initialize_matrix(i, N, a1, h_a);

  //initialize_matrix(1, N, a2, h_a);
  //initialize_matrix(2, N, a3, h_a);

  //copys variable to GPU
  cudaMemcpy(d_a, h_a, num_mat * N * N * sizeof(double), cudaMemcpyHostToDevice);
  cudaMemcpy(d_v,h_v, num_mat * N * N * sizeof(double), cudaMemcpyHostToDevice);
  cudaMemcpy(d_d, h_d, num_mat *N * sizeof(double), cudaMemcpyHostToDevice);


  clock_t cpu_startTime, cpu_endTime;
  double cpu_ElapseTime=0;
  cpu_startTime = clock();

  //Launches Kernel
  //<<<N/BLOCK_SIZE,BLOCK_SIZE>>>
  je << < (num_mat + nTPB - 1) / nTPB, nTPB >>> (num_mat, N,d_a, max_iter, d_v, d_d);

  cpu_endTime = clock();
  cpu_ElapseTime = ((cpu_endTime - cpu_startTime)/(double)CLOCKS_PER_SEC);
  std::cout << "CPU Elapsed time: " <<cpu_ElapseTime<<"s\n";


  //Copy back to CPU
  cudaMemcpy(h_d, d_d, num_mat * N * sizeof(double), cudaMemcpyDeviceToHost);


  //for(int i = 0; i< num_mat ; i++)
  //  print_vec(i, N, h_d);

  print_vec(0, N, h_d);
  //print_vec(2, N, h_d);

  return 0;
}

CPU Elapsed time: 0.000141s
matrix 0 Diagonal Values: 
0: -19.6210363083
1: -15.4706012068
2: -14.551543528
3: -9.92445127949
4: -9.23957205651
5: -8.5009620366
6: -4.22417814048
7: -3.05481075577
8: -0.778180509097
9: -0.574558603969
10: 2.28460733269
11: 2.90119749
12: 4.04045846922
13: 6.8042720621
14: 11.5708652454
15: 12.3955467015
16: 13.5322376081
17: 15.9642349456
18: 17.4585507174
19: 88.4526138531




In [8]:
%%script bash

for i in {1}; do 
g++ JacobiEigen_Serial/Jacobi_Eigen_Serial.cpp -o JacobiEigen_Serial/Jacobi_Eigen_Serial
./JacobiEigen_Serial/Jacobi_Eigen_Serial 20 0 real; done

CPU Elapsed time: 0.000333s


## 2. Execute .cu file via commands

### On Terminal:
```
nvcc -o Jacobi_Eigen_Parallel Jacobi_Eigen_Parallel.cu
cuda-memcheck ./Jacobi_Eigen_Parallel
```

### On Notebook:
```
!nvcc -o Jacobi_Eigen_Parallel Jacobi_Eigen_Parallel.cu
!cuda-memcheck ./Jacobi_Eigen_Parallel
```

In [9]:
!nvcc -o Jacobi_Eigen_Parallel Jacobi_Eigen_Parallel.cu
!cuda-memcheck ./Jacobi_Eigen_Parallel

========= CUDA-MEMCHECK
CPU Elapsed time: 0.034484s
matrix 0 Diagonal Values: 
0: -19.6210363083
1: -15.4706012068
2: -14.551543528
3: -9.92445127949
4: -9.23957205651
5: -8.5009620366
6: -4.22417814048
7: -3.05481075577
8: -0.778180509097
9: -0.574558603969
10: 2.28460733269
11: 2.90119749
12: 4.04045846922
13: 6.8042720621
14: 11.5708652454
15: 12.3955467015
16: 13.5322376081
17: 15.9642349456
18: 17.4585507174
19: 88.4526138531

========= ERROR SUMMARY: 0 errors


In [10]:
#!unzip Jacobi_Eigen.zip

In [11]:
%%script bash

for i in {1}; do 
g++ JacobiEigen_Serial/Jacobi_Eigen_Serial.cpp -o JacobiEigen_Serial/Jacobi_Eigen_Serial
./JacobiEigen_Serial/Jacobi_Eigen_Serial 20 0 real; done

CPU Elapsed time: 0.000318s
